# Resampling

You need to know how well your algorithms perform on unseen data. So far we have used all the data we have in our datasets to train and test our model.  However, this is not an accurate representation of our algorithms performance. When these models are deployed in production, we will not know how well it performs until new data that we have not used to train our model becomes available. At this point however, we have not labelled the data (it just arrived) and we would need to inspect it manually and assign the correct target label, and then check 

The best way to evaluate the performance of an algorithm would be to make predictions for new data to which you already know the answers. 

The second-best way is to use clever techniques from statistics called resampling methods that allow you to make accurate estimates for how well your algorithm will perform on new data. 

In this section you will discover how you can estimate the accuracy of your machine learning algorithms using resampling methods in Python and scikit-learn on the Pima Indians dataset. 

The evaluation is an estimate that we can use to talk about how well we think the algorithm may actually do in practice. It is not a guarantee of performance. Once we estimate the performance of our algorithm, we can then re-train the final algorithm on the entire training dataset and get it ready for operational use. 

### Resampling approaches
We are going to look at four different techniques that we can use to split up our training dataset and create useful estimates of performance for our machine learning algorithms:

- Train and Test Sets
- k-fold Cross-Validation
- Leave One Out Cross-Validation
- Repeated Random Test-Train Splits.

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

filename = './data/pima-indians-diabetes.data.csv'
header = ['Pregnancy_Count','Glucone_conc','Blood_pressure','Skin_thickness','Insulin','BMI','DPF','Age','Class']

data = pd.read_csv(filename, names=header)

array = data.values

# separate array into input features (X) and target output (Y) components
X = array[:,0:8]
Y = array[:,8]

In [2]:
data.head(5)

,Pregnancy_Count,Glucone_conc,Blood_pressure,Skin_thickness,Insulin,BMI,DPF,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Split into Train and Test Sets
The simplest method that we can use to evaluate the performance of a machine learning algorithm is to use different training and testing datasets. 

We can take our original dataset and split it into two parts. Train the algorithm on the first part, make predictions on the second part and evaluate the predictions against the expected results. The size of the split can depend on the size and specifics of your dataset, although it is common to use 67% of the data for training and the remaining 33% for testing.

This algorithm evaluation technique is very fast. It is ideal for large datasets (millions of records) where there is strong evidence that both splits of the data are representative of the underlying problem. Because of the speed, it is useful to use this approach when the algorithm you are investigating is slow to train. 

<div class="alert alert-info" role="alert">
    <div class="row vertical-align">
        <div class="col-xs-1 text-center">
            <i class="fa fa-exclamation-triangle fa-2x"/>
        </div>
        <div class="col-xs-11">
            <strong>
One downside of this technique is that it can have a high variance. This means that differences in the training and test dataset can result in meaningful differences in the estimate of accuracy. 
            </strong>
        </div>
    </div>
</div>

In the example below we split the Pima Indians dataset into 67%/33% splits for training and test and evaluate the accuracy of a Logistic Regression model.

In [3]:
from sklearn.model_selection import train_test_split

test_size = 0.33
seed = 7

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

model = LogisticRegression(solver = 'liblinear')
model.fit(X_train, Y_train)

result = model.score(X_test, Y_test)

print("Accuracy: %.3f%%" % (result*100.0))

Accuracy: 75.591%


### K-fold Cross-Validation

Cross-validation is an approach that you can use to estimate the performance of a machine learning algorithm with less variance than a single train-test set split. It works by splitting the dataset into k-parts (e.g. k=5 or k=10). Each split of the data is called a **fold**. 

The algorithm is trained on *k − 1* folds with one held back and tested on the held back fold. This is repeated so that each fold of the dataset is given a chance to be the held back test set. After running cross-validation you end up with *k* different performance scores that you can summarise using a **mean** and a **standard deviation**.

The result is a more reliable estimate of the performance of the algorithm on new data. It is more accurate because the algorithm is trained and evaluated multiple times on different data. The choice of *k* must allow the size of each test partition to be large enough to be a reasonable sample of the problem, whilst allowing enough repetitions of the train-test evaluation of the algorithm to provide a fair estimate of the algorithms performance on unseen data. 

For modest sized datasets in the thousands or tens of thousands of records, *k* values of 3, 5 and 10 are common. In the example below we use 10-fold cross-validation

In [4]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

num_folds = 10
seed = 7

kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)

model = LogisticRegression(solver = 'liblinear')

results = cross_val_score(model, X, Y, cv=kfold)
                          
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean() * 100.0, results.std() * 100.0))

Accuracy: 77.086% (5.091%)


### Leave One Out Cross-Validation

You can configure cross-validation so that the size of the fold is 1 (*k* is set to the number of observations in your dataset). This variation of cross-validation is called **leave-one-out** cross-validation. The result is a large number of performance measures that can be summarised in an effort to give a more reasonable estimate of the accuracy of your model on unseen data. 

<div class="alert alert-info" role="alert">
    <div class="row vertical-align">
        <div class="col-xs-1 text-center">
            <i class="fa fa-exclamation-triangle fa-2x"/>
        </div>
        <div class="col-xs-11">
            <strong>
A downside is that it can be a computationally more expensive procedure than k-fold cross-validation. 
            </strong>
        </div>
    </div>
</div>

In the example below we use leave-one-out cross-validation:

In [5]:
from sklearn.model_selection import LeaveOneOut

loocv = LeaveOneOut()

model = LogisticRegression(solver = 'liblinear')

results = cross_val_score(model, X, Y, cv=loocv)

print("Accuracy: %.3f%% (%.3f%%)" % (results.mean() * 100.0, results.std() * 100.0))

Accuracy: 76.823% (42.196%)


### Repeated Random Test-Train Splits

Another variation on k-fold cross-validation is to create a random split of the data like the train/test split described above, but repeat the process of splitting and evaluation of the algorithm multiple times, like cross-validation. 

This has the speed of using a train/test split and the reduction in variance in the estimated performance of k-fold cross-validation. You can also repeat the process many more times as needed to improve the accuracy. 

<div class="alert alert-info" role="alert">
    <div class="row vertical-align">
        <div class="col-xs-1 text-center">
            <i class="fa fa-exclamation-triangle fa-2x"/>
        </div>
        <div class="col-xs-11">
            <strong>
A down side is that repetitions may include much of the same data in the train or the test split from run to run, which introduces redundancy into the evaluation. 
            </strong>
        </div>
    </div>
</div>

The example below splits the data into a 67%/33% train/test split and repeats the process 10 times.

In [7]:
# Evaluate using Shuffle Split Cross Validation
from sklearn.model_selection import ShuffleSplit

n_splits = 10
test_size = 0.33
seed = 7

kfold = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=seed)

model = LogisticRegression(solver = 'liblinear')

results = cross_val_score(model, X, Y, cv=kfold)

print("Accuracy: %.3f%% (%.3f%%)" % (results.mean() * 100.0, results.std() * 100.0))

Accuracy: 76.535% (1.691%)


# Conclusion

Generally k-fold cross-validation is the gold standard for evaluating the performance of a machine learning algorithm on unseen data with *k* set to 3, 5, or 10.

- Using a train/test split is good for speed when using a slow algorithm and produces performance estimates with lower bias when using large datasets.
- Techniques like leave-one-out cross-validation and repeated random splits can be useful intermediates when trying to balance variance in the estimated performance, model training speed and dataset size.

The best advice is to experiment and find a technique for your problem that is fast and produces reasonable estimates of performance that you can use to make decisions. 

If in doubt, use **10-fold cross-validation**.